In [11]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies, DataSource, FrontMatter, FormulaTable, PageTabs, 
    HierarchyPaths, FormulaTableTypeSimple, FormulaTableTypeCalculus)
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies())
formulas_by_year = Formulas(data_source.formulas_by_year())
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative())

In [12]:


formulas_indices = formulas_by_year.filter_by_function(lambda x: x['Category'] == 'Indices')

calculus_summary = FormulaTableTypeCalculus(formulas_by_year)

display(calculus_summary.to_dataframe())



,Derivative,Equivalent integral,Comment
0,"$ y=f(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\...",,
1,"$ y=kx \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\...",,
2,"$ y=k \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,...",,
3,"$ y=x^n \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,...",,
4,"$ y=kx^n \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\...",,
5,"$ y=kf(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,...",,
6,"$ y=f(x) + g(x) \,\,\,\,\,\,\,\,\,\,\,\,\,\,...",,
7,"$y=f(x)^n \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,...",$ {\Large\int} f'(x)[f(x)]^n dx = \dfrac{1}{n+...,When n = -1 follow the integration rules for ...
8,"$y=uv \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,...",$ {\Large\int} u \dfrac{dv}{dx} dx=uv-{\Large\...,
9,"$y=\dfrac{u}{v} \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,...",,
